# Economy Health Heatmap
Visualizes economic indicators to predict Fed's stance on interest rates.
Colors range from Red (weak) to Green (strong) based on within-year percentile rankings.

In [1]:
import bql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Initialize BQL
bq = bql.Service()

ModuleNotFoundError: No module named 'bql'

In [ ]:
# Economic Indicator Definitions
# Dictionary maps display name -> (Bloomberg ticker, is_higher_better)
# is_higher_better: True = higher values indicate stronger economy
#                   False = lower values indicate stronger economy (e.g., unemployment)

INDICATORS = {
    # Employment (strong labor market = hawkish Fed)
    'NFP': ('NFP TCH Index', True),              # Non-Farm Payrolls - higher = more jobs
    'Initial Claims': ('INJCJC Index', False),   # Jobless claims - lower = fewer layoffs
    'Cont. Claims': ('INJCSP Index', False),     # Continuing claims - lower = better
    'UR': ('USURTOT Index', False),              # Unemployment rate - lower = stronger
    'ADP': ('ADP CHNG Index', True),             # ADP jobs - higher = more hiring
    'JOLTS Total': ('JOLTTOTL Index', True),     # Job openings - higher = strong demand
    
    # Growth & Activity (strong growth = hawkish Fed)
    'Retail Services YoY': ('RSTAYOY Index', True),       # Retail sales growth
    'Industrial Production YoY': ('IP YOY Index', True),  # Manufacturing output
    'Durable Goods': ('DGNOCHNG Index', True),            # Durable goods orders
    'GDP QoQ': ('GDP CQOQ Index', True),                  # GDP growth
    
    # Surveys & Sentiment (optimism = hawkish Fed)
    'ISM Manufacturing': ('NAPMPMI Index', True),   # Manufacturing PMI
    'ISM Services': ('NAPMNMI Index', True),        # Services PMI
    'NAHB': ('USHBMIDX Index', True),               # Homebuilder sentiment
    'Michigan Sentiment': ('CONSSENT Index', True), # Consumer sentiment
    'Consumer Confidence': ('CONCCONF Index', True), # Conference Board
    
    # Inflation (higher inflation = hawkish Fed)
    'Core CPI MoM': ('CPUPXCHG Index', True),   # Core CPI month-over-month
    'PCE MoM': ('PCE CMOM Index', True),        # PCE month-over-month
    'CPI Absolute': ('CPI XYOY Index', True),   # CPI year-over-year
    'PCE Absolute': ('PCE CYOY Index', True),   # PCE year-over-year
}

# Category groupings for display
CATEGORIES = {
    'Employment': ['NFP', 'Initial Claims', 'Cont. Claims', 'UR', 'ADP', 'JOLTS Total'],
    'Growth': ['Retail Services YoY', 'Industrial Production YoY', 'Durable Goods', 'GDP QoQ'],
    'Surveys': ['ISM Manufacturing', 'ISM Services', 'NAHB', 'Michigan Sentiment', 'Consumer Confidence'],
    'Inflation': ['Core CPI MoM', 'PCE MoM', 'CPI Absolute', 'PCE Absolute']
}

In [ ]:
def fetch_indicator_data(ticker, years_back=3):
    """
    Fetch historical data for an economic indicator using BQL.
    Returns DataFrame with Date and Value columns.
    """
    try:
        end_date = datetime.now()
        start_date = end_date - timedelta(days=years_back * 365)
        
        # Create BQL request for economic data
        request = bql.Request(
            ticker,
            {
                'Value': bq.data.px_last()['value'],
            },
            with_params={
                'fill': 'prev',
                'dates': bq.func.range(
                    start_date.strftime('%Y-%m-%d'),
                    end_date.strftime('%Y-%m-%d')
                )
            }
        )
        
        response = bq.execute(request)
        df = pd.concat([item.df() for item in response], axis=1)
        df = df.reset_index()
        df.columns = ['Date', 'Value']
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.dropna()
        
        return df
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return None

In [ ]:
def calculate_yearly_percentile(df, is_higher_better=True):
    """
    Calculate percentile rank within each year.
    Returns value between 0 and 1.
    
    If is_higher_better=True: higher values get higher percentiles (closer to 1)
    If is_higher_better=False: lower values get higher percentiles (inverted)
    """
    df = df.copy()
    df['Year'] = df['Date'].dt.year
    
    # Calculate percentile rank within each year
    df['Percentile'] = df.groupby('Year')['Value'].rank(pct=True)
    
    # If lower is better (like unemployment), invert the percentile
    if not is_higher_better:
        df['Percentile'] = 1 - df['Percentile']
    
    return df

In [ ]:
def fetch_all_indicators():
    """
    Fetch data for all indicators and return latest values with percentile rankings.
    """
    results = {}
    
    print("Fetching economic indicator data...\n")
    
    for name, (ticker, is_higher_better) in INDICATORS.items():
        print(f"  Fetching {name}...", end=" ")
        
        df = fetch_indicator_data(ticker)
        
        if df is not None and len(df) > 0:
            df = calculate_yearly_percentile(df, is_higher_better)
            
            # Get latest data point
            latest = df.iloc[-1]
            
            results[name] = {
                'value': latest['Value'],
                'percentile': latest['Percentile'],
                'date': latest['Date'],
                'year': latest['Year'],
                'is_higher_better': is_higher_better,
                'history': df
            }
            print(f"OK (Latest: {latest['Value']:.2f}, Percentile: {latest['Percentile']:.1%})")
        else:
            print("FAILED")
    
    return results

In [ ]:
# Fetch all indicator data
indicator_data = fetch_all_indicators()

In [ ]:
def create_heatmap_data(indicator_data):
    """
    Create DataFrame for heatmap visualization.
    Organized by category with percentile values.
    """
    rows = []
    
    for category, indicators in CATEGORIES.items():
        for indicator in indicators:
            if indicator in indicator_data:
                data = indicator_data[indicator]
                rows.append({
                    'Category': category,
                    'Indicator': indicator,
                    'Value': data['value'],
                    'Percentile': data['percentile'],
                    'Date': data['date'].strftime('%Y-%m-%d')
                })
    
    return pd.DataFrame(rows)

In [ ]:
def plot_economy_heatmap(indicator_data):
    """
    Create a heatmap showing economic health.
    Red = Weak (dovish Fed), Green = Strong (hawkish Fed)
    """
    # Prepare data
    df = create_heatmap_data(indicator_data)
    
    if len(df) == 0:
        print("No data available for heatmap")
        return
    
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Create custom colormap: Red -> Yellow -> Green
    from matplotlib.colors import LinearSegmentedColormap
    colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
    cmap = LinearSegmentedColormap.from_list('RdYlGn', colors)
    
    # Organize data by category
    category_order = ['Employment', 'Growth', 'Surveys', 'Inflation']
    df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)
    df = df.sort_values(['Category', 'Indicator'])
    
    # Create matrix for heatmap (single column for current percentile)
    indicators = df['Indicator'].tolist()
    percentiles = df['Percentile'].values.reshape(-1, 1)
    values = df['Value'].tolist()
    
    # Plot heatmap
    im = ax.imshow(percentiles, cmap=cmap, aspect='auto', vmin=0, vmax=1)
    
    # Configure axes
    ax.set_yticks(range(len(indicators)))
    ax.set_yticklabels(indicators, fontsize=11)
    ax.set_xticks([0])
    ax.set_xticklabels(['Current'], fontsize=12)
    
    # Add category labels
    current_category = None
    category_positions = []
    
    for i, (_, row) in enumerate(df.iterrows()):
        if row['Category'] != current_category:
            category_positions.append((i, row['Category']))
            current_category = row['Category']
    
    # Add category divider lines
    for pos, _ in category_positions[1:]:
        ax.axhline(y=pos - 0.5, color='white', linewidth=3)
    
    # Annotate cells with actual values and percentiles
    for i, (val, pct) in enumerate(zip(values, percentiles.flatten())):
        text_color = 'white' if pct < 0.3 or pct > 0.7 else 'black'
        ax.text(0, i, f'{val:.1f}\n({pct:.0%})', 
                ha='center', va='center', fontsize=10, 
                color=text_color, fontweight='bold')
    
    # Add category labels on left side
    ax2 = ax.twinx()
    ax2.set_ylim(ax.get_ylim())
    ax2.set_yticks([])
    
    for pos, cat in category_positions:
        cat_indicators = CATEGORIES[cat]
        cat_count = len([x for x in cat_indicators if x in indicators])
        mid_pos = pos + (cat_count - 1) / 2
        ax.text(-0.6, mid_pos, cat, ha='right', va='center', 
                fontsize=12, fontweight='bold', rotation=0)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, shrink=0.8, pad=0.02)
    cbar.set_label('Economic Strength (Year Percentile)', fontsize=11)
    cbar.set_ticks([0, 0.25, 0.5, 0.75, 1.0])
    cbar.set_ticklabels(['Weak\n(Dovish)', '25%', 'Neutral', '75%', 'Strong\n(Hawkish)'])
    
    plt.title('Economy Health Heatmap\nFed Policy Stance Predictor', 
              fontsize=16, fontweight='bold', pad=20)
    
    fig.text(0.5, 0.02, 
             'Green = Strong economy (hawkish Fed, rates stay high/rise) | Red = Weak economy (dovish Fed, rate cuts likely)',
             ha='center', fontsize=10, style='italic')
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.08)
    plt.show()
    
    return df

In [ ]:
# Generate the heatmap
summary_df = plot_economy_heatmap(indicator_data)

In [ ]:
def calculate_fed_stance_score(indicator_data):
    """
    Calculate overall Fed stance prediction based on all indicators.
    Returns a score from -1 (very dovish) to +1 (very hawkish).
    """
    # Weights by category (what matters most to the Fed)
    category_weights = {
        'Employment': 0.30,   # Fed dual mandate - employment
        'Inflation': 0.35,    # Fed dual mandate - price stability
        'Growth': 0.20,       # Economic activity
        'Surveys': 0.15       # Forward-looking sentiment
    }
    
    category_scores = {}
    
    for category, indicators in CATEGORIES.items():
        percentiles = []
        for indicator in indicators:
            if indicator in indicator_data:
                percentiles.append(indicator_data[indicator]['percentile'])
        
        if percentiles:
            category_scores[category] = np.mean(percentiles)
    
    # Calculate weighted overall score
    overall_score = 0
    total_weight = 0
    
    for category, score in category_scores.items():
        weight = category_weights.get(category, 0.25)
        overall_score += score * weight
        total_weight += weight
    
    if total_weight > 0:
        overall_score /= total_weight
    
    # Convert to -1 to +1 scale (0.5 percentile = 0 score)
    fed_stance = (overall_score - 0.5) * 2
    
    return fed_stance, category_scores, overall_score

In [ ]:
def plot_fed_stance_gauge(indicator_data):
    """
    Create a gauge visualization showing predicted Fed stance.
    """
    fed_stance, category_scores, overall = calculate_fed_stance_score(indicator_data)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left: Category breakdown
    ax1 = axes[0]
    categories = list(category_scores.keys())
    scores = [category_scores[c] for c in categories]
    colors = ['#d73027' if s < 0.35 else '#fee08b' if s < 0.65 else '#1a9850' for s in scores]
    
    bars = ax1.barh(categories, scores, color=colors, edgecolor='black', linewidth=1.5)
    ax1.axvline(x=0.5, color='black', linestyle='--', linewidth=2, label='Neutral')
    ax1.set_xlim(0, 1)
    ax1.set_xlabel('Strength Score (0 = Weak, 1 = Strong)', fontsize=11)
    ax1.set_title('Economic Strength by Category', fontsize=13, fontweight='bold')
    
    for bar, score in zip(bars, scores):
        ax1.text(score + 0.02, bar.get_y() + bar.get_height()/2,
                f'{score:.1%}', va='center', fontsize=11, fontweight='bold')
    
    # Right: Overall Fed stance
    ax2 = axes[1]
    
    gauge_colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
    gauge_labels = ['Very\nDovish', 'Dovish', 'Lean\nDovish', 'Lean\nHawkish', 'Hawkish', 'Very\nHawkish']
    
    for i, (color, label) in enumerate(zip(gauge_colors, gauge_labels)):
        ax2.barh(0, 1/6, left=i/6, color=color, height=0.5)
        ax2.text((i + 0.5)/6, -0.4, label, ha='center', va='top', fontsize=9)
    
    marker_pos = (fed_stance + 1) / 2
    ax2.plot(marker_pos, 0, marker='v', markersize=20, color='black')
    ax2.plot(marker_pos, 0, marker='v', markersize=15, color='yellow')
    
    ax2.set_xlim(0, 1)
    ax2.set_ylim(-1, 0.5)
    ax2.set_yticks([])
    ax2.set_xticks([])
    ax2.set_title(f'Predicted Fed Stance: {fed_stance:+.2f}', fontsize=13, fontweight='bold')
    
    if fed_stance > 0.3:
        interpretation = "Economy running hot - expect rates to stay elevated"
    elif fed_stance > 0:
        interpretation = "Economy moderately strong - limited room for cuts"
    elif fed_stance > -0.3:
        interpretation = "Economy softening - rate cuts becoming possible"
    else:
        interpretation = "Economy weakening - rate cuts likely"
    
    ax2.text(0.5, -0.8, interpretation, ha='center', va='top', 
             fontsize=11, style='italic', wrap=True)
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*60)
    print("FED STANCE ANALYSIS SUMMARY")
    print("="*60)
    print(f"\nOverall Economic Strength: {overall:.1%}")
    print(f"Fed Stance Score: {fed_stance:+.2f} (-1=Dovish, +1=Hawkish)")
    print(f"\nCategory Breakdown:")
    for cat, score in category_scores.items():
        status = "Strong" if score > 0.6 else "Weak" if score < 0.4 else "Neutral"
        print(f"  {cat:<15} {score:.1%} ({status})")
    print(f"\nInterpretation: {interpretation}")
    print("="*60)

In [ ]:
# Generate Fed stance analysis
plot_fed_stance_gauge(indicator_data)

In [ ]:
def plot_historical_heatmap(indicator_data, months_back=12):
    """
    Create a time-series heatmap showing how indicators evolved over time.
    """
    all_data = []
    
    for name, data in indicator_data.items():
        if 'history' in data:
            df = data['history'].copy()
            df['Indicator'] = name
            df['Month'] = df['Date'].dt.to_period('M')
            
            monthly = df.groupby('Month').last().reset_index()
            monthly = monthly.tail(months_back)
            all_data.append(monthly[['Month', 'Indicator', 'Percentile']])
    
    if not all_data:
        print("No historical data available")
        return
    
    combined = pd.concat(all_data, ignore_index=True)
    pivot = combined.pivot(index='Indicator', columns='Month', values='Percentile')
    
    indicator_order = []
    for cat in CATEGORIES.values():
        for ind in cat:
            if ind in pivot.index:
                indicator_order.append(ind)
    
    pivot = pivot.reindex(indicator_order)
    
    fig, ax = plt.subplots(figsize=(16, 10))
    
    from matplotlib.colors import LinearSegmentedColormap
    colors = ['#d73027', '#fc8d59', '#fee08b', '#d9ef8b', '#91cf60', '#1a9850']
    cmap = LinearSegmentedColormap.from_list('RdYlGn', colors)
    
    sns.heatmap(pivot, cmap=cmap, vmin=0, vmax=1, 
                annot=True, fmt='.0%', annot_kws={'size': 8},
                cbar_kws={'label': 'Economic Strength (Percentile)'},
                ax=ax)
    
    ax.set_xticklabels([str(m) for m in pivot.columns], rotation=45, ha='right')
    
    plt.title(f'Economic Indicators Evolution - Last {months_back} Months\n'
              f'Green = Strong (Hawkish Fed) | Red = Weak (Dovish Fed)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Month', fontsize=11)
    plt.ylabel('Indicator', fontsize=11)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Generate historical heatmap
plot_historical_heatmap(indicator_data, months_back=12)

In [ ]:
# Summary table of current readings
print("\n" + "="*80)
print("CURRENT ECONOMIC INDICATOR READINGS")
print("="*80 + "\n")

for category, indicators in CATEGORIES.items():
    print(f"\n{category.upper()}")
    print("-" * 50)
    
    for indicator in indicators:
        if indicator in indicator_data:
            data = indicator_data[indicator]
            pct = data['percentile']
            
            if pct >= 0.7:
                signal = "STRONG"
            elif pct >= 0.5:
                signal = "ABOVE AVG"
            elif pct >= 0.3:
                signal = "BELOW AVG"
            else:
                signal = "WEAK"
            
            print(f"  {indicator:<25} {data['value']:>10.2f}  ({pct:>5.1%})  [{signal}]")

print("\n" + "="*80)